In [1]:
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage import morphology, filters, measure
from skimage.morphology import disk, area_opening
from scipy.ndimage import median_filter
from sklearn.metrics import jaccard_score
from skimage.measure import regionprops
import skimage
from scans import *

In [2]:
def plot_slice(slice, title):
    plt.imshow(slice, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()
    
def load_data(scan_path, label_path):
    scan = nib.load(scan_path).get_fdata()
    label = nib.load(label_path).get_fdata()
    return scan, label

def select_slice(scan, label, slice_idx):
    return scan[:, :, slice_idx], label[:, :, slice_idx]

def dice_coefficient(segmented, ground_truth):
    intersection = np.logical_and(segmented, ground_truth)
    return (2. * intersection.sum()) / (segmented.sum() + ground_truth.sum())

# Morphological Operations
def morphological_operations(image):
    selem = disk(2)
    eroded = morphology.erosion(image, selem)
    return eroded

# Image Filtering
def image_filtering(image):
    return median_filter(image, size=5)

# Threshold Segmentation
def threshold_segmentation(image):
    threshold = filters.threshold_otsu(image)
    binary = image > threshold
    return binary

# Small Object Removal
def small_object_removal(image, min_size=100):
    return morphology.remove_small_objects(image, min_size)

# Modify Small Object Removal
def selective_object_removal(image):
    labeled_image, num_labels = skimage.measure.label(image, connectivity=2, return_num=True)
    
    # Find the largest connected component
    largest_region = None
    max_region_area = 0
    for region in regionprops(labeled_image):
        if region.area > max_region_area:
            max_region_area = region.area
            largest_region = region
            
    # Create a new image where only the largest connected component is retained
    cleaned_image = np.zeros_like(image)
    if largest_region:
        minr, minc, maxr, maxc = largest_region.bbox
        cleaned_image[minr:maxr, minc:maxc] = largest_region.image
    
    return cleaned_image

def image_subtraction(image1, image2):
    return np.bitwise_xor(image1, image2).astype(np.uint8)

In [3]:
dataset_path = 'D:\Dosyalar\dataset\Task06_Lung'
images_train_dir = os.path.join(dataset_path, 'imagesTr')
labels_train_dir = os.path.join(dataset_path, 'labelsTr')

In [4]:
global_dscs = []
global_sensitivities = []
global_specificities = []
global_accuracies = []
global_weights = []
global_TP = 0
global_TN = 0
global_FP = 0
global_FN = 0

In [5]:
with open('results.txt', 'w') as results_file:
    for scan_number in INDEXES[-21:]:
        scan_path = os.path.join(images_train_dir, f'lung_{scan_number}.nii.gz')
        label_path = os.path.join(labels_train_dir, f'lung_{scan_number}.nii.gz')
        scan, label = load_data(scan_path, label_path)
        
        slice_indexes = globals()[f'scan_{scan_number}']

        for i, slice_idx in enumerate(slice_indexes):
            slice, slice_label = select_slice(scan, label, slice_idx)

            morph_image = morphological_operations(slice)
            filtered_image = image_filtering(morph_image)
            thresh_image = threshold_segmentation(filtered_image)
            cleaned_image = selective_object_removal(thresh_image)
            segmented_mask = image_subtraction(cleaned_image, thresh_image)

            TP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 1))
            TN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 0))
            FP = np.sum(np.logical_and(segmented_mask == 1, slice_label == 0))
            FN = np.sum(np.logical_and(segmented_mask == 0, slice_label == 1))

            global_TP += TP
            global_TN += TN
            global_FP += FP
            global_FN += FN

            dsc = dice_coefficient(segmented_mask, slice_label)
            print(f'Scan {scan_number} - Slice {slice_idx}: {dsc}')
            sensitivity = TP / (TP + FN) if TP + FN > 0 else 0
            specificity = TN / (TN + FP) if TN + FP > 0 else 0
            accuracy = (TP + TN) / (TP + TN + FP + FN) if TP + TN + FP + FN > 0 else 0

            global_dscs.append(dsc)
            global_sensitivities.append(sensitivity)
            global_specificities.append(specificity)
            global_accuracies.append(accuracy)
            global_weights.append(np.sum(segmented_mask))

Scan 062 - Slice 145: 0.0
Scan 062 - Slice 146: 0.10632911392405063
Scan 062 - Slice 147: 0.4354587869362364
Scan 062 - Slice 148: 0.5016722408026756
Scan 062 - Slice 149: 0.5612244897959183
Scan 062 - Slice 150: 0.639412997903564
Scan 062 - Slice 151: 0.7264864864864865
Scan 062 - Slice 152: 0.6902119071644803
Scan 062 - Slice 153: 0.7795823665893271
Scan 062 - Slice 154: 0.7141162514827996
Scan 062 - Slice 155: 0.6635071090047393
Scan 062 - Slice 156: 0.5518102372034956
Scan 062 - Slice 157: 0.5092748735244519
Scan 062 - Slice 158: 0.5098039215686274
Scan 062 - Slice 159: 0.09049773755656108
Scan 062 - Slice 160: 0.0
Scan 062 - Slice 161: 0.0
Scan 064 - Slice 119: 0.0
Scan 064 - Slice 120: 0.028824833702882482
Scan 064 - Slice 121: 0.4348308374930671
Scan 064 - Slice 122: 0.5522151898734177
Scan 064 - Slice 123: 0.8202197802197803
Scan 064 - Slice 124: 0.8906367041198502
Scan 064 - Slice 125: 0.9043241402791965
Scan 064 - Slice 126: 0.8961352657004831
Scan 064 - Slice 127: 0.0
Scan 0

In [6]:
# Calculate overall weighted mean and standard deviation of metrics
overall_weighted_mean_dsc = np.average(global_dscs, weights=global_weights)
overall_mean_dsc = np.mean(global_dscs) 
overall_weighted_std_dsc = np.sqrt(np.average((global_dscs - overall_weighted_mean_dsc) ** 2, weights=global_weights))
overall_weighted_mean_sensitivity = np.average(global_sensitivities, weights=global_weights)
overall_weighted_std_sensitivity = np.sqrt(np.average((global_sensitivities - overall_weighted_mean_sensitivity) ** 2, weights=global_weights))
overall_weighted_mean_specificity = np.average(global_specificities, weights=global_weights)
overall_weighted_std_specificity = np.sqrt(np.average((global_specificities - overall_weighted_mean_specificity) ** 2, weights=global_weights))
overall_weighted_mean_accuracy = np.average(global_accuracies, weights=global_weights)
overall_weighted_std_accuracy = np.sqrt(np.average((global_accuracies - overall_weighted_mean_accuracy) ** 2, weights=global_weights))
# Calculate overall confusion matrix percentages
true_positive_rate = global_TP / (global_TP + global_FN) * 100
true_negative_rate = global_TN / (global_TN + global_FP) * 100
false_positive_rate = global_FP / (global_FP + global_TN) * 100
false_negative_rate = global_FN / (global_FN + global_TP) * 100

In [7]:
overall_mean_dsc = np.mean(global_dscs)
overall_mean_sensitivity = np.mean(global_sensitivities)
overall_mean_specificity = np.mean(global_specificities)
overall_mean_accuracy = np.mean(global_accuracies)
print("Overall Mean DSC:", overall_mean_dsc)
print("Overall Mean Sensitivity:", overall_mean_sensitivity)
print("Overall Mean Specificity:", overall_mean_specificity)
print("Overall Mean Accuracy:", overall_mean_accuracy)


Overall Mean DSC: 0.14231672822870514
Overall Mean Sensitivity: 0.1361715925451325
Overall Mean Specificity: 0.9981512767333696
Overall Mean Accuracy: 0.9956316625126557


In [7]:
print(f'Overall Weighted Mean DSC: {overall_weighted_mean_dsc}, Standard Deviation: {overall_weighted_std_dsc}')
print(f'Overall Mean DSC: {overall_mean_dsc}')
print(f'Overall Weighted Mean Sensitivity: {overall_weighted_mean_sensitivity}, Standard Deviation: {overall_weighted_std_sensitivity}')
print(f'Overall Weighted Mean Specificity: {overall_weighted_mean_specificity}, Standard Deviation: {overall_weighted_std_specificity}')
print(f'Overall Weighted Mean Accuracy: {overall_weighted_mean_accuracy}, Standard Deviation: {overall_weighted_std_accuracy}')

Overall Weighted Mean DSC: 0.17731379552065074, Standard Deviation: 0.2506032724779433
Overall Mean DSC: 0.14231672822870514
Overall Weighted Mean Sensitivity: 0.21918939261445466, Standard Deviation: 0.2323063711926806
Overall Weighted Mean Specificity: 0.9935162714690619, Standard Deviation: 0.009399542380414372
Overall Weighted Mean Accuracy: 0.9913031417815584, Standard Deviation: 0.009320896437291082


In [8]:
print(f'True Positive Rate: {true_positive_rate}%')
print(f'True Negative Rate: {true_negative_rate}%')
print(f'False Positive Rate: {false_positive_rate}%')
print(f'False Negative Rate: {false_negative_rate}%')

True Positive Rate: 16.389672348308952%
True Negative Rate: 99.81502345623564%
False Positive Rate: 0.18497654376435685%
False Negative Rate: 83.61032765169105%
